## Loading Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
import os

In [5]:
BASE_PATH = './data/'
os.listdir(BASE_PATH)

['movie.csv']

In [ ]:
df = pd.read_csv(f'{BASE_PATH}/movie.csv', encoding='utf-8')
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
df.describe()

,label
count,40000.000000
mean,0.499525
std,0.500006
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [ ]:
df.isnull()

,text,label
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
39995,False,False
39996,False,False
39997,False,False
39998,False,False


In [ ]:
df.columns

Index(['text', 'label'], dtype='object')

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train_x, test_x, y_train, y_test = train_test_split(df.text,df.label, test_size=0.25,random_state=123)

In [ ]:
#TF-IDF
tfv = TfidfVectorizer(min_df=3,  max_features=None, strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                    ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1, stop_words = 'english')

tfv.fit(list(train_x) + list(test_x))
train_x_tfv =  tfv.transform(train_x) 
test_x_tfv = tfv.transform(test_x)

In [ ]:
train_x.shape

(30000,)

In [ ]:
y_train.shape

(30000,)

In [ ]:
test_x.shape, y_test.shape

((10000,), (10000,))

In [ ]:
dataset = tf.data.experimental.make_csv_dataset('/content/movie.csv', batch_size=32,
                                                select_columns=['\ufefftext', 'label'],
                                                label_name='label')
print(dataset.take(1))
iterator = dataset.as_numpy_iterator()
# print(dict(next(iterator)))

<TakeDataset element_spec=(OrderedDict([('\ufefftext', TensorSpec(shape=(32,), dtype=tf.string, name=None))]), TensorSpec(shape=(32,), dtype=tf.int32, name=None))>


In [ ]:
dataset = dataset.map(lambda x, y: (tf.concat([tf.expand_dims(x['\ufefftext'], axis=-1)], axis=-1), y))

## Building the model

In [ ]:
model = tf.keras.Sequential([
    # layers.Input(shape=(30000, )),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile('adam', tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
model.fit(dataset, epochs=10)

Epoch 1/10


UnimplementedError: ignored